# 02504 - Exam

## Helper Functions


In [50]:
import numpy as np
import cv2 
import os
import scipy
import matplotlib.pyplot as plt
import matplotlib

def load_im(path : str, greyscale : bool = False) -> np.ndarray:
    """
        Takes:
            :param path: Path to where the image should be loaded
            :param greyscale: A flag that determines whether the image should be loaded as greyscale or not
            
        Returns:
            Image scaled to float.
    """
    im = cv2.imread(path)[:, :, ::-1]
    im = im.astype(np.float64) / 255
    
    if greyscale:
        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    
    return im


def pi(points : np.ndarray) -> np.ndarray:
    """
        Converts from homogeneous to inhomogeneous coordinates
    """
    p = points[:-1]/points[-1]
    
    return p


def piInv(points : np.ndarray) -> np.ndarray:
    """
        Converts from inhomogeneous to homogeneous coordinates
    """
    
    # Gets the amount of points by using shape
    _, num_points = points.shape
    
    # Stacks the scale s at the bottom of the matrix
    ph = np.vstack((points, np.ones(num_points)))
    
    return ph


def projectPoints(K, Rt, Q):
    
    Q_hom = piInv(Q)
    points = K @ Rt @ Q_hom
    points_inhom = pi(points)
    
    return points_inhom


def hest(q1, q2) -> np.ndarray:
    """
        Takes two points in 2D and returns the estimated homography matrix.
    """
    
    if len(q1) != len(q2):
        raise ValueError("There must be an equal amount of points in the two sets!")
    
    Bi = []
    for i in range(q1.shape[1]):
        qi = q1[:,i]   # <-- getting the first column
        
        # Creating that weird qx matrix for the Kronecker product
        q1x = np.array(
            [[0,        -1, qi[1]],
             [1,        0, -qi[0]],
             [-qi[1], qi[0], 0]]
        )
        
        q2t_hom = q2[:, i].reshape(-1, 1) # <-- getting the first column and reshaping does for dim: (1, ) -> (1,1)
        Bi.append(np.kron(q2t_hom.T, q1x)) # <-- formula follows that of week 2, slide 56
        # print(np.kron(q2t_hom.T, q1x).shape)
       
    B = np.concatenate(Bi, axis=0)
    
    # Some TA prooved that it was unneseccary to find their dot product
    #BtB = B.T @ B
    V, Lambda, Vt = np.linalg.svd(B)
    Ht = Vt[-1, :]
    
    Ht = np.reshape(Ht, (3, 3))
    H = Ht.T
    
    return H
    
    
def crossOp(p : np.ndarray) -> np.ndarray:
    """
        One of Them weird functions. It takes in a 3D vector and then returns
        some gnarly matrix.
    """
    p = p.flatten()
    if p.size != 3:
        raise Exception("Invalid input, vector must be exactly 3D.")
    
    x, y, z = p
    px = np.array(
        [[0, -z, y],
         [z, 0, -x],
         [-y, x, 0]]
    )
    
    return px


def computeFundamentalMatrix(K1 : np.ndarray, K2 : np.ndarray, R2 : np.ndarray, t2 : np.ndarray) -> np.ndarray:
    """
        Computing the fundamental matrix between two camera matrices K1 & K2.
    """
    t2x = crossOp(t2)

    E = t2x @ R2

    K1inv = np.linalg.inv(K1)
    K2inv = np.linalg.inv(K2)

    F = K1inv.T @ E @ K2inv
    
    return F


def fancyRotate(theta_x, theta_y, theta_z):
    """
        Does the rotation matrix that we have seen a few times.
        E.g. Exercises week 4, eq(12).
    """
    from scipy.spatial.transform import Rotation
    
    R = Rotation.from_euler("xyz", [theta_x, theta_y, theta_z]).as_matrix()
    
    return R


def in_frame(l, l_im, shape):
    """
        I think this checks whether the line is within the image
    """
    q = np.cross(l.flatten(), l_im)
    q = q[:2]/q[2]
    if all(q >= 0) and all(q+1 <= shape[1::-1]):
        return q
    
    
def DrawLine(l, shape):
    """
        Checks where the line intersects the four sides of the image
        and finds the two intersections that are within the frame
    """
    lines = [[1, 0, 0], [0, 1, 0], [1, 0, 1-shape[1]], [0, 1, 1-shape[0]]]
    P = [in_frame(l, l_im, shape) for l_im in lines if in_frame(l, l_im, shape) is not None]
    plt.plot(*np.array(P).T)


def draw_line_Vitus(line: np.ndarray, tau: float):
    """
        Draws a line with a width of tau.
        
        Takes:
            :param line: The line to be drawn
            :param tau: The distance from the line to be drawn
        
        Returns:
            :return: None
    """
    
    x1, x2 = plt.gca().get_xlim()
    y1 = -(line[0]*x1 + line[2]) / line[1]
    y2 = -(line[0]*x2 + line[2]) / line[1]
    y1_low = -(line[0]*x1 + line[2] + tau) / line[1]
    y1_high = -(line[0]*x1 + line[2] - tau) / line[1]
    y2_low = -(line[0]*x2 + line[2] + tau) / line[1]
    y2_high = -(line[0]*x2 + line[2] - tau) / line[1]
    
    plt.axline((x1,y1), (x2,y2), c='black')
    plt.axline((x1,y1_low), (x2,y2_low), c='black', linestyle='dashed')
    plt.axline((x1,y1_high), (x2,y2_high), c='black', linestyle='dashed')


def triangulate(q_thicc : list, P_thicc : list):
    """
        Should take in:
            A list of n pixel-coordinates: [q1, q2, ..., qn]
            
            A list of n projection matrices: [P1, P2, ..., Pn]
        
        And return:
            The triangulation of the 3D point by utilizing the linear algorithm.
    """
    
    n = len(P_thicc)
    m = P_thicc[0].shape[1]
    
    B = np.zeros((2*n, m))
    
    for i in range(n):
        Pi = P_thicc[i]
        x, y = q_thicc[i]
        x, y = x.item(), y.item()   # <-- apparently there could be some issues with indexing of arrays
        
        B[i*2] = Pi[2] * x - Pi[0]
        B[i * 2 + 1] = Pi[2] * y - Pi[1]
        
    u, s, vh = np.linalg.svd(B)
    v = vh.T
    Q = v[:, -1]
    
    Q = Q.T / Q[-1] # <-- This scaling was highly recommended by Andreas <3
    
    return Q


def RMSE(q : np.ndarray, q_tilde : np.ndarray) -> np.ndarray:
    m = q.shape[1]
    reproject_err = np.sqrt(np.sum(np.power(q_tilde - q, 2)) / m)
    return reproject_err


def checkerboardPoints(n : int, m : int) -> np.ndarray:
    """
        Takes:
            an integer: n
            
            an integer: m
        
        Returns:
            Weird matrix of size 3 x (n * m) : Q
            (As defined per week 4 exercises, eq (7))
        
        Idiot code* explained:
            We can't numpy.hstack to an empty array, so we initialize the first column, then
            when we return, we just return all of the matrix except for the first column.
    """
    
    Q = np.zeros((3, 1))    # <-- idiot code (1/2)
    
    for i in range(n):
        for j in range(m):
            temp = np.array([i - (n - 1)/2, j - (m - 1)/2, 0]).reshape(-1, 1)
            Q = np.hstack((Q, temp))
    
    Q = Q[:, 1:]    # <-- iditot code (2/2)
    
    return Q

## Actual exam

In [51]:
PATH = "../Data/Exam/"


### Quest 1

$
\textcolor{magenta}{\text{Check this question again but about distortion coefficients!}}
$

In [52]:
def camera_matrix(f, deltax, deltay, alpha, beta):
    K =  np.array(
        [[f, beta*f, deltax],
        [0, alpha*f, deltay],
        [0, 0, 1]]
    )
    
    return K

In [53]:
f = 1200
principal_point = (400, 350)
deltaX, deltaY = principal_point
alpha = 1
beta = 0
k3 = 0.01
k5 = 0.04

K = camera_matrix(f, deltaX, deltaY, alpha, beta)

print("\nCamera Matrix:\n", K)



Camera Matrix:
 [[1200    0  400]
 [   0 1200  350]
 [   0    0    1]]


### Quest 2

Intuitivly I would think the answer is *f* but after reading lecture notes it would seem that $\alpha$ is a scaling parameter, hence I would assume that it should simply be halfed.

In [54]:
resolution = (800, 600)

K2 = np.array([[1000, 0, 400],
              [0, 1000, 300],
              [0, 0, 1]])

new_size = (400, 300)



In [55]:
# Camera matrix K2 has resolution 800x600, we wish to resize it to 400x300
# We can do this by multiplying K2 with a scaling matrix S
S = np.array([[0.5, 0, 0],
              [0, 0.5, 0],
              [0, 0, 1]])

K2_S = K2 @ S

print("\nCamera Matrix:\n", K2_S)


Camera Matrix:
 [[500.   0. 400.]
 [  0. 500. 300.]
 [  0.   0.   1.]]


### Quest 3


In [56]:
point_3D = np.array([-0.03, 0.01, 0.59]).reshape(-1, 1)

print(piInv(point_3D))

[[-0.03]
 [ 0.01]
 [ 0.59]
 [ 1.  ]]


In [57]:
f = 1720
principal_point = (680, 610)
deltaX, deltaY = principal_point
alpha = 1
beta = 0

R = cv2.Rodrigues(np.array([-0.1, -0.1, -0.2]))[0]

t = np.array([[0.09], [0.05], [0.05]])

point_3D = np.array([-0.03, 0.01, 0.59]).reshape(-1, 1)

K3 = camera_matrix(f, deltaX, deltaY, alpha, beta)
Rt3 = np.hstack((R, t))
projectedPoint = projectPoints(K3, Rt3, point_3D)

print(f"projectedPoints: \n{projectedPoint}")



projectedPoints: 
[[707.94410717]
 [964.4581973 ]]


### Quest 5

In [58]:
K = np.array(
    [
        [900, 0, 1070],
        [0, 900, 610.0],
        [0, 0, 1]
    ],
    float)
R1 = cv2.Rodrigues(np.array([-1.6, 0.3, -2.1]))[0]
t1 = np.array([[0.0], [1.0], [3.0]], float)
R2 = cv2.Rodrigues(np.array([-0.4, -1.3, -1.6]))[0]
t2 = np.array([[0.0], [1.0], [6.0]], float)
R3 = cv2.Rodrigues(np.array([2.5, 1.7, -0.4]))[0]
t3 = np.array([[2.0], [-7.0], [25.0]], float)

Rt1 = np.hstack((R1, t1))
Rt2 = np.hstack((R2, t2))
Rt3 = np.hstack((R3, t3))

p1 = np.array([[1046.0], [453.0]])
p2 = np.array([[1126.0], [671.0]])
p3 = np.array([[1165.0], [453.0]])

In [59]:
P1 = K @ Rt1
P2 = K @ Rt2
P3 = K @ Rt3

qs = [p1, p2, p3]
Ps = [P1, P2, P3]

Q_1 = triangulate(qs, Ps)

print(f"Q_1: \n{Q_1}\n")
Q_1 = Q_1.reshape(-1, 1)
print(f"Q_1 reshaped: \n{Q_1}")

Q_1: 
[3.10058867 0.74321098 0.46490561 1.        ]

Q_1 reshaped: 
[[3.10058867]
 [0.74321098]
 [0.46490561]
 [1.        ]]


In [60]:
# Point in camera 2
q2 = K @ Rt2 @ Q_1

t2x = crossOp(t2)

E2 = t2x @ R2

Kinv = np.linalg.inv(K)



F2 = Kinv.T @ E2 @ Kinv

# No! Cause we need to use F, since we are in homogenous coordinates
print(f"This is the epipolar line! \n{F2 @ q2}")

This is the epipolar line! 
[[-5.20082784e-02]
 [-1.61649515e-02]
 [ 6.79342210e+01]]


### Quest 6


In [62]:
print(f"Triangulated point: \n{Q_1}")

Triangulated point: 
[[3.10058867]
 [0.74321098]
 [0.46490561]
 [1.        ]]


### Quest 11

In [66]:
# Homogenous point
p_hom = np.array([2, 4, 3]).reshape(-1, 1)

# Inhomogenous point
p_inhom = pi(p_hom)

# Line
l_hom = np.array([1, 2, 2]).reshape(-1, 1)

# inhomogenous line
l_inhom = pi(l_hom)

# Finding the distance from line to point
d = np.abs(l_inhom.T @ p_inhom) / np.sqrt(l_inhom[0]**2 + l_inhom[1]**2)

print(f"Distance: {d.item()}")

Distance: 1.4907119849998596


In [76]:
# I think it might just be the dot product:
l_hom_scaled = l_hom / np.linalg.norm(l_hom[:2])
print(f"Line scaled: \n{l_hom_scaled}\n")

d2 = np.abs(l_hom_scaled.T @ p_hom)

print(f"Distance 2: {d2.item()}")

Line scaled: 
[[0.4472136 ]
 [0.89442719]
 [0.89442719]]

Distance 2: 7.155417527999328


### Quest 12

#### (Additional) Helper Functions

In [77]:
def gaussian1DKernel(sigma: float, SDs: int = 3) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
        Takes in:
            The standard deviation: sigma
            
            (optional - 3 as default) The amount of standard deviations: SDs
            (From statistics we know that 3 standard deviations encapsulates 99.7% of the Gaussian distribution)
        
        Returns:
            The 1D Gaussian kernel: g
            
            The derivative: gd
    """
    
    # Morten was very big brains when he definde this kernel!
    bounds = round(SDs * sigma)
    x = np.arange(-bounds, bounds + 1)
    
    g_unorm = np.exp(- x**2/(2 * sigma**2))
    g = g_unorm / g_unorm.sum() # <-- makes sure that g sums to one (SUCH THAT IS A PROPABILITY DISTRIBUTION!)
    
    gd = -x / (sigma**2) * g
    
    return g, gd


def gaussianSmoothing(im : np.ndarray, sigma : float) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
        Takes:
            The image as a numpy array: im
            
            The standard deviation: sigma
        
        Returns:
            The smoothed image: I
            
            The smoothed x-derivative of the image: Ix
            
            The smoothed y-derivative of the image: Iy
    """
    
    g, gd = gaussian1DKernel(sigma)
    
    im_smooth = cv2.sepFilter2D(im, -1, g, g)
    
    Ix = cv2.sepFilter2D(im, -1, gd, g)
    
    Iy = cv2.sepFilter2D(im, -1, g, gd)
    
    return im_smooth, Ix, Iy


def smoothedHessian(im : np.ndarray, sigma : float, epsilon : int) -> np.ndarray:
    """
        Takes:
            The image as a numpy array: im
            
            The standard deviation: sigma
            
            The width of the Gaussian kernel: epsilon
        
        Returns:
            The smoothed hessian: C
    """
    
    g_eps, _ = gaussian1DKernel(epsilon, SDs = 1)
    
    _, Ix, Iy = gaussianSmoothing(im, sigma)
    
    a = cv2.sepFilter2D(Ix**2, -1, g_eps, g_eps)
    b = cv2.sepFilter2D(Iy**2, -1, g_eps, g_eps)
    c = cv2.sepFilter2D(Ix * Iy, -1, g_eps, g_eps)
    
    C = np.array(
        [[a, c],
         [c, b]]
    )
    
    return C


def harrisMeasure(im : np.ndarray, sigma : float, epsilon : int, k : float = 0.06) -> np.ndarray:
    """
        Takes:
            The image as a numpy array: im
            
            The standard deviation: sigma
            
            The width of the Gaussian kernel: epsilon
            
            (Optional - default is 0.06) Some real scaling factor: k
        
        Returns:
            The harris measure: r
    """
    
    C = smoothedHessian(im, sigma, epsilon)
    
    a = C[0, 0]
    b = C[1, 1]
    c = C[0, 1]
    
    #if c != C[1, 0]:
    #    raise ValueError("Something went wrong in the previous function, oh no!")
    
    r = a * b - c**2 - k * (a + b)**2
    
    return r


def cornerDetector(im : np.ndarray, sigma : float, epsilon : int, k : float = 0.06, tau : float = None) -> list:
    """
        Takes:
            The image as a numpy array: im
            
            The standard deviation: sigma
            
            The width of the Gaussian kernel: epsilon
            
            (optional - default a tenth of max value of harris meassure) Threshold: tau
            
            (Optional - default is 0.06) Some real scaling factor: k
            
        Returns:
            List of detected corners: c
    """
    
    
    r = harrisMeasure(im, sigma, epsilon)
    if tau is None:
        tau = 0.1 * np.max(r, axis = None)  # <-- defined as per week 6, slide 31
    
    mask = r > tau     # <-- boolean array
    
    # non-maximum suprresion
    mask[:-1, :] *= r[:-1, :] > r[1:, :] 
    mask[1:, :] *= r[1:, :] > r[:-1, :]
    mask[:, :-1] *= r[:, :-1] > r[:, 1:]
    mask[:, 1:] *= r[:, 1:] > r[:, :-1]
    
    mask[1:, 1:] *= r[1:, 1:] > r[:-1, :-1] 
    mask[:-1, :-1] *= r[:-1, :-1] > r[1:, 1:]
    
    c = np.where(mask)
    c = np.array([c[1], c[0]])  # < -- apparently np.where switches the two coordinates
    
    return c

#### Response

In [82]:
c = np.load(PATH + "harris.npy", allow_pickle=True).item()

gI2x = c["g*(I_x^2)"]
gI2y = c["g*(I_y^2)"]
gIxy = c["g*(I_x I_y)"]

print(f"gI2x: \n{gI2x}\n")
print(f"gI2y: \n{gI2y}\n")
print(f"gIyx: \n{gIxy}\n")

gI2x: 
[[16.8 18.5 20.  20.8 20.6]
 [21.  23.4 25.5 26.7 26.5]
 [25.8 29.  31.8 33.4 33.2]
 [30.4 34.4 37.9 39.9 39.7]
 [33.9 38.6 42.7 45.1 44.9]]

gI2y: 
[[35.2 31.8 27.1 22.  17.6]
 [33.3 30.2 25.9 21.4 17.4]
 [29.3 26.8 23.3 19.5 16.2]
 [24.4 22.5 19.8 16.9 14.6]
 [19.5 18.2 16.3 14.4 12.8]]

gIyx: 
[[-6.5 -6.3 -5.2 -3.3 -1. ]
 [-6.7 -6.9 -6.  -4.1 -1.6]
 [-6.5 -7.1 -6.4 -4.7 -2.3]
 [-5.9 -6.7 -6.3 -4.9 -2.8]
 [-4.8 -5.7 -5.7 -4.8 -3.2]]



In [87]:
a = gI2x
b = gI2y
c = gIxy

# honestly redundant:    
C = np.array(
        [[a, c],
         [c, b]]
    )

# Harris measure
k = 0.06
r = a * b - c**2 - k * (a + b)**2

# corner detection
tau = 516
if tau is None:
        tau = 0.1 * np.max(r, axis = None)  # <-- defined as per week 6, slide 31
    
mask = r > tau     # <-- boolean array

# non-maximum suprresion
mask[:-1, :] *= r[:-1, :] > r[1:, :] 
mask[1:, :] *= r[1:, :] > r[:-1, :]
mask[:, :-1] *= r[:, :-1] > r[:, 1:]
mask[:, 1:] *= r[:, 1:] > r[:, :-1]

mask[1:, 1:] *= r[1:, 1:] > r[:-1, :-1] 
mask[:-1, :-1] *= r[:-1, :-1] > r[1:, 1:]

print(mask)
c = np.where(mask)
print(c)
c = np.array([c[1], c[0]])  # < -- apparently np.where switches the two coordinates


print(c)


[[False False False False False]
 [False False False False False]
 [False  True False False False]
 [False False False False False]
 [False False False False False]]
(array([2], dtype=int64), array([1], dtype=int64))
[[1]
 [2]]


### Quest 13

#### (Additional) Helper Functions

In [90]:
def line_estimate(p1: np.ndarray, p2: np.ndarray) -> np.ndarray:
    """
        Takes:
            :param p1: A 2D point in homogenous coordinates
            :param p2: A 2D point in homogenous coordinates
        
        Returns:
            The line estimate between the two points
    """
    
    line_estimate = np.cross(p1, p2)
    
    # divide the first two coordinates by their own norm
    line_estimate = line_estimate / np.linalg.norm(line_estimate[:2])
    
    return line_estimate


def determine_inliers(l: np.ndarray, points: np.ndarray, tau: float) -> np.ndarray:
    """
        Takes:
            :param l: A line estimate in homogenous coordinates
            :param points: A set of points in homogenous coordinates
            :param tau: The threshold to determine whether a point is an inlier
        
        Returns:
            A boolean array of the same length as points, where True indicates that the point is an inlier
    """
    # Compute the distance between the line and the points
    distances = np.abs(l @ points)
    
    # Determine which points are within the threshold
    inliers = distances < tau
    
    
    return inliers


def consensus(l: np.ndarray, points: np.ndarray, tau: float) -> int:
    """
        Takes:
            :param l: A line estimate in homogenous coordinates
            :param points: A set of points in homogenous coordinates
            :param tau: The threshold to determine whether a point is an inlier
        
        Returns:
            The number of inliers
    """
    
    inliers = determine_inliers(l, points, tau)
    num_inliers = np.sum(inliers)
    
    return num_inliers

#### Response

In [122]:
ransac = np.load(PATH + "RANSAC.npy", allow_pickle=True).item()

tau = 0.2

points = ransac["points"]
x1 = ransac["x1"].reshape(-1, 1)
x2 = ransac["x2"].reshape(-1, 1)


print(f"x1: \n{x1}\n")
print(f"x2: \n{x2}\n")

x1 = piInv(x1).flatten()
x2 = piInv(x2).flatten()



# print(f"l_est: \n{np.cross(x1, x2)}\n")

print(f"x1 (hom): \n{x1}\n")
print(f"x2 (hom): \n{x2}\n")


l = line_estimate(x1, x2).reshape(-1, 1)
points_hom = piInv(points)

print(f"Line estimate Homogenous: \n{l}\n")

# Determine the number of inliers
inliers = consensus(l.T, points_hom, tau)

print(f"Inliers: {inliers}")

x1: 
[[0.48075454]
 [0.91547038]]

x2: 
[[0.58356452]
 [0.77721424]]

x1 (hom): 
[0.48075454 0.91547038 1.        ]

x2 (hom): 
[0.58356452 0.77721424 1.        ]

Line estimate Homogenous: 
[[ 0.80245086]
 [ 0.59671821]
 [-0.93205974]]

Inliers: 34


### Quest 14

In [130]:
# n = 2 for lines
# n = 4 for homographies
n = 4

curr_iter = 191
inliers = 103
matches = 404

frac = inliers / matches
p = 0.95

e_hat = 1 - frac

N_hat = np.ceil(np.log(1 - p)/(np.log(1 - (1 - e_hat)**n)))

print(N_hat)
print(curr_iter + N_hat)

708.0
899.0


In [124]:
print(np.log(1))

0.0


### Quest 18

In [149]:
import sympy
k11, k12, k13, k21, k22, k23, k31, k32, k33 = sympy.symbols('k11 k12 k13 k21 k22 k23 k31 k32 k33')

K = sympy.Matrix([[k11, k12, k13], [k21, k22, k23], [k31, k32, k33]])
R = sympy.eye(3)

KRT = K @ R 

print(KRT)

Matrix([[k11, k12, k13], [k21, k22, k23], [k31, k32, k33]])


### Quest 20

In [1]:
theta = 0
n = 8
F = 14
s = 16

x = np.array(range(F + 1))

y = 1/2 + (1/2) * np.cos(n * theta + 2*np.pi * x/s)

plt.scatter(x, y)
plt.show()

NameError: name 'np' is not defined